In [1]:
import os, sys, email
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
#import plotly
#plotly.offline.init_notebook_mode()
#import plotly.graph_objs as go
# import wordcloud

# Network analysis
import networkx as nx
# NLP
from nltk.tokenize.regexp import RegexpTokenizer

In [2]:
# Read the data into a DataFrame
emails_df = pd.read_csv('emails_enron.csv')
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [4]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

In [5]:
emails_df.head(10)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p
5,allen-p/_sent_mail/1002.,<30965995.1075863688265.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Greg,\n\n How about either next Tuesday or Thu...",allen-p
6,allen-p/_sent_mail/1003.,<16254169.1075863688286.JavaMail.evans@thyme>,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",(phillip.allen@enron.com),"(david.l.johnson@enron.com, john.shafer@enron....",,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,"david.l.johnson@enron.com, John Shafer",,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Please cc the following distribution list with...,allen-p
7,allen-p/_sent_mail/1004.,<17189699.1075863688308.JavaMail.evans@thyme>,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",(phillip.allen@enron.com),(joyce.teixeira@enron.com),Re: PRC review - phone calls,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Joyce Teixeira,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,any morning between 10 and 11:30,allen-p
8,allen-p/_sent_mail/101.,<20641191.1075855687472.JavaMail.evans@thyme>,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",(phillip.allen@enron.com),(mark.scott@enron.com),Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,1. login: pallen pw: ke9davis\n\n I don't thi...,allen-p
9,allen-p/_sent_mail/102.,<30795301.1075855687494.JavaMail.evans@thyme>,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",(phillip.allen@enron.com),(zimam@enron.com),FW: fixed forward or other Collar floor gas pr...,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,zimam@enron.com,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,allen-p


In [6]:
print(emails_df.loc[0]["X-From"])

Phillip K Allen


In [7]:
emails_df2 = emails_df[['Subject', 'X-cc', 'X-bcc']]

In [8]:
emails_df2[1:10]

,Subject,X-cc,X-bcc
1,Re:,,
2,Re: test,,
3,,,
4,Re: Hello,,
5,Re: Hello,,
6,,,
7,Re: PRC review - phone calls,,
8,Re: High Speed Internet Access,,
9,FW: fixed forward or other Collar floor gas pr...,,


In [10]:
emails_df2[300005:300010]

,Subject,X-cc,X-bcc
300005,Bufete Industrial busca alianza con Corporativ...,,
300006,Re: Bravo,,
300007,Re: Conference Room,"John Moore, Mark Bernstein, Robert Sevitz, Sco...",
300008,Re: Bravo,,
300009,RE: WEEKLY GE CONFERENCE CALL,,


In [11]:
emails_df2 = emails_df2.apply(lambda x: x.astype(str).str.lower())

In [12]:
emails_df2[300005:300010]

,Subject,X-cc,X-bcc
300005,bufete industrial busca alianza con corporativ...,,
300006,re: bravo,,
300007,re: conference room,"john moore, mark bernstein, robert sevitz, sco...",
300008,re: bravo,,
300009,re: weekly ge conference call,,


In [13]:
emails_df2 = emails_df2.replace('', 'sent')
emails_df2

,Subject,X-cc,X-bcc
0,sent,sent,sent
1,re:,sent,sent
2,re: test,sent,sent
3,sent,sent,sent
4,re: hello,sent,sent
5,re: hello,sent,sent
6,sent,sent,sent
7,re: prc review - phone calls,sent,sent
8,re: high speed internet access,sent,sent
9,fw: fixed forward or other collar floor gas pr...,sent,sent


In [14]:
emails_df2[300005:300010]

,Subject,X-cc,X-bcc
300005,bufete industrial busca alianza con corporativ...,sent,sent
300006,re: bravo,sent,sent
300007,re: conference room,"john moore, mark bernstein, robert sevitz, sco...",sent
300008,re: bravo,sent,sent
300009,re: weekly ge conference call,sent,sent


In [15]:
stat = emails_df2.Subject.str[:3]
emails_df2['status'] = np.where(stat == 're:', 're',
                       np.where(stat == 'fw:', 'fw', 'sent'))  

In [16]:
emails_df2[300005:300010]

,Subject,X-cc,X-bcc,status
300005,bufete industrial busca alianza con corporativ...,sent,sent,sent
300006,re: bravo,sent,sent,re
300007,re: conference room,"john moore, mark bernstein, robert sevitz, sco...",sent,re
300008,re: bravo,sent,sent,re
300009,re: weekly ge conference call,sent,sent,re


In [17]:
emails_df2

,Subject,X-cc,X-bcc,status
0,sent,sent,sent,sent
1,re:,sent,sent,re
2,re: test,sent,sent,re
3,sent,sent,sent,sent
4,re: hello,sent,sent,re
5,re: hello,sent,sent,re
6,sent,sent,sent,sent
7,re: prc review - phone calls,sent,sent,re
8,re: high speed internet access,sent,sent,re
9,fw: fixed forward or other collar floor gas pr...,sent,sent,fw


In [19]:
emails_df2 = emails_df2.rename(columns={'X-cc': 'xcc', 'X-bcc': 'xbcc'})

In [20]:
emails_df2

,Subject,xcc,xbcc,status
0,sent,sent,sent,sent
1,re:,sent,sent,re
2,re: test,sent,sent,re
3,sent,sent,sent,sent
4,re: hello,sent,sent,re
5,re: hello,sent,sent,re
6,sent,sent,sent,sent
7,re: prc review - phone calls,sent,sent,re
8,re: high speed internet access,sent,sent,re
9,fw: fixed forward or other collar floor gas pr...,sent,sent,fw


In [21]:
c = emails_df2.xcc.str[:4]
emails_df2['cc'] = np.where(c == 'sent', 0, 'cc')

In [23]:
emails_df2[517371:517380]

,Subject,xcc,xbcc,status,cc
517371,re:,'jzufferli@shaw.ca',sent,re,cc
517372,training,sent,sent,sent,0
517373,re: w/c deficiency rider,sent,sent,re,0
517374,re: lethbrige ironworks,sent,sent,re,0
517375,lethbrige ironworks,'katrina@valeopower.com',sent,sent,cc
517376,re: quick question,sent,sent,re,0
517377,re: new company - online trader access (stack ...,sent,sent,re,0
517378,gas and power eol access,sent,sent,sent,0
517379,re: market risk meeting today - john zufferli,sent,sent,re,0


In [24]:
b = emails_df2.xbcc.str[:4]
emails_df2['bcc'] = np.where(b == 'sent', 0, 'bcc')

In [26]:
emails_df2[517371:517380]

,Subject,xcc,xbcc,status,cc,bcc
517371,re:,'jzufferli@shaw.ca',sent,re,cc,0
517372,training,sent,sent,sent,0,0
517373,re: w/c deficiency rider,sent,sent,re,0,0
517374,re: lethbrige ironworks,sent,sent,re,0,0
517375,lethbrige ironworks,'katrina@valeopower.com',sent,sent,cc,0
517376,re: quick question,sent,sent,re,0,0
517377,re: new company - online trader access (stack ...,sent,sent,re,0,0
517378,gas and power eol access,sent,sent,sent,0,0
517379,re: market risk meeting today - john zufferli,sent,sent,re,0,0


In [27]:
emails_df2.drop(columns=['Subject', 'xcc', 'xbcc'])

,status,cc,bcc
0,sent,0,0
1,re,0,0
2,re,0,0
3,sent,0,0
4,re,0,0
5,re,0,0
6,sent,0,0
7,re,0,0
8,re,0,0
9,fw,0,0


In [28]:
cond = emails_df2.cc == 'cc'
emails_df2.status[cond] = emails_df2.cc[cond]

In [30]:
emails_df2[517371:517380]

,Subject,xcc,xbcc,status,cc,bcc
517371,re:,'jzufferli@shaw.ca',sent,cc,cc,0
517372,training,sent,sent,sent,0,0
517373,re: w/c deficiency rider,sent,sent,re,0,0
517374,re: lethbrige ironworks,sent,sent,re,0,0
517375,lethbrige ironworks,'katrina@valeopower.com',sent,cc,cc,0
517376,re: quick question,sent,sent,re,0,0
517377,re: new company - online trader access (stack ...,sent,sent,re,0,0
517378,gas and power eol access,sent,sent,sent,0,0
517379,re: market risk meeting today - john zufferli,sent,sent,re,0,0


In [31]:
cond2 = emails_df2.bcc == 'bcc'
emails_df2.status[cond2] = emails_df2.bcc[cond]

In [33]:
emails_df2.drop(columns=['Subject', 'xcc', 'xbcc'])

,status,cc,bcc
0,sent,0,0
1,re,0,0
2,re,0,0
3,sent,0,0
4,re,0,0
5,re,0,0
6,sent,0,0
7,re,0,0
8,re,0,0
9,fw,0,0


In [34]:
emails_df2[517371:517380]

,Subject,xcc,xbcc,status,cc,bcc
517371,re:,'jzufferli@shaw.ca',sent,cc,cc,0
517372,training,sent,sent,sent,0,0
517373,re: w/c deficiency rider,sent,sent,re,0,0
517374,re: lethbrige ironworks,sent,sent,re,0,0
517375,lethbrige ironworks,'katrina@valeopower.com',sent,cc,cc,0
517376,re: quick question,sent,sent,re,0,0
517377,re: new company - online trader access (stack ...,sent,sent,re,0,0
517378,gas and power eol access,sent,sent,sent,0,0
517379,re: market risk meeting today - john zufferli,sent,sent,re,0,0


In [35]:
emails_df2[emails_df2['status'].str.contains("bcc")==True]

,Subject,xcc,xbcc,status,cc,bcc
5386,move date,"shively, hunter s. </o=enron/ou=na/cn=recipien...","quezada, daniel </o=enron/ou=na/cn=recipients/...",bcc,cc,bcc
18162,fundamental overview presentation,none,none,bcc,cc,bcc
18178,midwest/southeast desk meeting w/fletch sturm,none,none,bcc,cc,bcc
18179,congestion in the transmission system for t...,none,none,bcc,cc,bcc
18414,call laddie for house party:,none,none,bcc,cc,bcc
20506,call laddie for house party:,none,none,bcc,cc,bcc
25327,luncheon meeting to revise revenue model,"choate, heather </o=enron/ou=na/cn=recipients/...","beck, sally </o=enron/ou=na/cn=recipients/cn=s...",bcc,cc,bcc
25334,fw: truequote testing and live date,"beck, sally </o=enron/ou=na/cn=recipients/cn=s...","denny, jennifer </o=enron/ou=na/cn=recipients/...",bcc,cc,bcc
25350,sally beck has moved!,"mcloughlin, hector </o=enron/ou=na/cn=recipien...","beck, sally </o=enron/ou=na/cn=recipients/cn=s...",bcc,cc,bcc
29166,prc attendee lists,"beck, sally </o=enron/ou=na/cn=recipients/cn=s...","choate, heather </o=enron/ou=na/cn=recipients/...",bcc,cc,bcc


In [37]:
emails_df2[18162:18180]

,Subject,xcc,xbcc,status,cc,bcc
18162,fundamental overview presentation,none,none,bcc,cc,bcc
18163,east power ubs start-up project - coordination...,"calger, christopher f. </o=enron/ou=na/cn=reci...",sent,cc,cc,0
18164,"playa del carmen, mex.",sent,sent,sent,0,0
18165,"dove hunt in mason, tx",sent,sent,sent,0,0
18166,ees / ews / epmi meeting - midwest retail load...,sent,sent,sent,0,0
18167,shiner bocktoberfest:,sent,sent,sent,0,0
18168,mark doggett shower @ iowa colo,sent,sent,sent,0,0
18169,sent,sent,sent,sent,0,0
18170,doggett bach party @ crystal beach,sent,sent,sent,0,0
18171,dove hunt in mason tx - joe ebert,sent,sent,sent,0,0


In [38]:
emails_df2.drop(columns=['Subject', 'xcc', 'xbcc', 'cc', 'bcc'])

,status
0,sent
1,re
2,re
3,sent
4,re
5,re
6,sent
7,re
8,re
9,fw


In [55]:
emails_df3 = emails_df[['Date', 'From', 'To', 'X-From', 'X-To', 'content', 'user']]

In [56]:
enron_df = pd.concat([emails_df3, emails_df2], axis=1)

In [57]:
enron_df[15:18]

,Date,From,To,X-From,X-To,content,user,Subject,xcc,xbcc,status,cc,bcc
15,"Thu, 5 Oct 2000 05:55:00 -0700 (PDT)",(phillip.allen@enron.com),(paula.harris@enron.com),Phillip K Allen,Paula Harris,"Paula,\n\n 35 million is fine\n\nPhillip",allen-p,re: 2001 margin plan,sent,sent,re,0,0
16,"Wed, 4 Oct 2000 09:23:00 -0700 (PDT)",(phillip.allen@enron.com),(ina.rangel@enron.com),Phillip K Allen,Ina Rangel,---------------------- Forwarded by Phillip K ...,allen-p,"var, reporting and resources meeting",sent,sent,sent,0,0
17,"Fri, 4 May 2001 11:26:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.heizenrader@enron.com),Phillip K Allen,Tim Heizenrader <Tim Heizenrader/Enron@EnronXG...,"Tim,\n\nmike grigsby is having problems with a...",allen-p,sent,sent,sent,sent,0,0


In [58]:
enron_df = enron_df.rename(columns={'Date': 'date', 'From': 'email_from', 'To': 'email_to', 'X-From': 'from', 'X-To': 'to', 'Subject': 'title'})

In [59]:
enron_df[20:25]

,date,email_from,email_to,from,to,content,user,title,xcc,xbcc,status,cc,bcc
20,"Tue, 3 Oct 2000 09:13:00 -0700 (PDT)",(phillip.allen@enron.com),(bs_stone@yahoo.com),Phillip K Allen,bs_stone@yahoo.com,"Brenda,\n\nPlease use the second check as the ...",allen-p,sent,sent,sent,sent,0,0
21,"Tue, 3 Oct 2000 03:17:00 -0700 (PDT)",(phillip.allen@enron.com),(stouchstone@natsource.com),Phillip K Allen,Steve Touchstone <STouchstone@natsource.com>,I think Fletch has a good CPA. I am still doi...,allen-p,re: not business related..,sent,sent,re,0,0
22,"Mon, 2 Oct 2000 02:19:00 -0700 (PDT)",(phillip.allen@enron.com),(bs_stone@yahoo.com),Phillip K Allen,"""BS Stone"" <bs_stone@yahoo.com> @ ENRON","Brenda,\n\n Please use the second check as my ...",allen-p,re: original sept check/closing,sent,sent,re,0,0
23,"Thu, 28 Sep 2000 06:17:00 -0700 (PDT)",(phillip.allen@enron.com),(lkuch@mh.com),Phillip K Allen,Lkuch@mh.com,---------------------- Forwarded by Phillip K ...,allen-p,san juan index,sent,sent,sent,0,0
24,"Thu, 28 Sep 2000 05:56:00 -0700 (PDT)",(phillip.allen@enron.com),(jeffrey.hodge@enron.com),Phillip K Allen,Jeffrey T Hodge,"Liane,\n\n As we discussed yesterday, I am con...",allen-p,san juan index,sent,sent,sent,0,0


In [64]:
enron_df.iloc[1]

date                       Fri, 4 May 2001 13:51:00 -0700 (PDT)
email_from                            (phillip.allen@enron.com)
email_to                              (john.lavorato@enron.com)
from                                            Phillip K Allen
to            John J Lavorato <John J Lavorato/ENRON@enronXg...
content       Traveling to have a business meeting takes the...
user                                                    allen-p
title                                                       re:
xcc                                                        sent
xbcc                                                       sent
status                                                       re
cc                                                            0
bcc                                                           0
Name: 1, dtype: object

In [65]:
enron_emails_df = enron_df[['title', 'date', 'from', 'to', 'status', 'content']]

In [66]:
enron_emails_df[20:30]

,title,date,from,to,status,content
20,sent,"Tue, 3 Oct 2000 09:13:00 -0700 (PDT)",Phillip K Allen,bs_stone@yahoo.com,sent,"Brenda,\n\nPlease use the second check as the ..."
21,re: not business related..,"Tue, 3 Oct 2000 03:17:00 -0700 (PDT)",Phillip K Allen,Steve Touchstone <STouchstone@natsource.com>,re,I think Fletch has a good CPA. I am still doi...
22,re: original sept check/closing,"Mon, 2 Oct 2000 02:19:00 -0700 (PDT)",Phillip K Allen,"""BS Stone"" <bs_stone@yahoo.com> @ ENRON",re,"Brenda,\n\n Please use the second check as my ..."
23,san juan index,"Thu, 28 Sep 2000 06:17:00 -0700 (PDT)",Phillip K Allen,Lkuch@mh.com,sent,---------------------- Forwarded by Phillip K ...
24,san juan index,"Thu, 28 Sep 2000 05:56:00 -0700 (PDT)",Phillip K Allen,Jeffrey T Hodge,sent,"Liane,\n\n As we discussed yesterday, I am con..."
25,investment structure,"Tue, 26 Sep 2000 09:28:00 -0700 (PDT)",Phillip K Allen,kholst@enron.com,sent,---------------------- Forwarded by Phillip K ...
26,investment structure,"Tue, 26 Sep 2000 09:26:00 -0700 (PDT)",Phillip K Allen,pallen70@hotmail.com,sent,---------------------- Forwarded by Phillip K ...
27,sent,"Tue, 26 Sep 2000 07:01:00 -0700 (PDT)",Phillip K Allen,Ina Rangel,sent,---------------------- Forwarded by Phillip K ...
28,sent,"Thu, 3 May 2001 15:57:00 -0700 (PDT)",Phillip K Allen,rlehmann <rlehmann@yahoo.com>,sent,"Reagan,\n\nJust wanted to give you an update. ..."
29,re: gas trading vision meeting,"Tue, 26 Sep 2000 05:11:00 -0700 (PDT)",Phillip K Allen,Cindy Cicchetti,re,Nymex expiration is during this time frame. P...
